In [5]:
import h5py
import pickle
import numpy as np
import glob
import os
import torch
import random

output_dir = r"C:\Users\uhewm\Desktop\ProjectHGT\simulation_chunks"
all_files = sorted(glob.glob(os.path.join(output_dir, "*.h5")))

x = []
theta = []

for file in random.sample(all_files, 100):
#for file in all_files:
    with h5py.File(file, "r") as f:
        grp = f["results"]
        
        graph_properties = pickle.loads(grp["graph_properties"][()])
        # Tensors erstellen
        nodes = torch.tensor(graph_properties[0])          # Shape [199]
        edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
        coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]

        #coords[0] = coords[0] / max(coords[0]) * max(coords[1])
        
        # Falls nötig, auffüllen
        if edges.shape[1] < nodes.shape[0]:
            padding = torch.full((2, nodes.shape[0] - edges.shape[1]), -1, dtype=edges.dtype)
            edges = torch.cat([edges, padding], dim=1)     # Jetzt Shape [2, 199]
        
        # Alles zu einem Tensor kombinieren: z. B. pro Knoten eine Zeile mit:
        # [node_id, coord_x, coord_y, edge_from, edge_to]
        combined_x = torch.stack([nodes, coords[0], coords[1], edges[0], edges[1]], dim=1)  # Shape [199, 5]
        x.append(combined_x)

        theta_gains = torch.tensor([1 if node in grp.attrs["parental_nodes_hgt_events_corrected"] else 0 for node in graph_properties[0]])
        theta_losses = torch.tensor([1 if node in grp.attrs["children_gene_nodes_loss_events"] else 0 for node in graph_properties[0]])
        #combined_theta = torch.stack([theta_gains, theta_losses], dim = 1)
        combined_theta = torch.stack([theta_gains], dim = 1)
        theta.append(combined_theta)

# Für SNPE vorbereiten
x_all = torch.stack(x).float()
theta_all = torch.stack(theta).float()

x_all_flat = x_all.view(x_all.size(0), -1)

C:\Users\uhewm\AppData\Local\Temp\ipykernel_21512\1768722487.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
C:\Users\uhewm\AppData\Local\Temp\ipykernel_21512\1768722487.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]


In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sbi import utils
from sbi.inference import SNPE
from sbi.neural_nets import posterior_nn


import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCNEmbedding(torch.nn.Module):
    def __init__(self, in_channels=2, hidden_channels=32, gcn_out_channels=32, final_out_channels=1):
        super().__init__()

        self.num_nodes = 199         # feste Anzahl Knoten (z.B.)
        self.features_per_node = 5   # wie bei combined tensor

        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, gcn_out_channels)
        
        self.fc_out = torch.nn.Linear(gcn_out_channels, final_out_channels)  # 2 binäre Outputs pro Knoten

    def forward(self, x):
        B = x.size(0)  # Batchgröße
        outputs = []

        for i in range(B):
            sample_flat = x[i]  # [num_nodes * features_per_node]
            sample = sample_flat.view(self.num_nodes, self.features_per_node)
            
            nodes = sample[:, 0].long()
            node_id_to_idx = {nid.item(): idx for idx, nid in enumerate(nodes)}
            
            raw_edges = sample[:, 3:5].T.long()
            #edges_mapped = torch.tensor([
            #    [node_id_to_idx.get(n.item(), -1) for n in raw_edges[0]],
            #    [node_id_to_idx.get(n.item(), -1) for n in raw_edges[1]],
            #])

            mask = (raw_edges[0] >= 0) & (raw_edges[1] >= 0)
            edge_index = raw_edges[:, mask]

            # Optional: ungerichtet machen
            #edge_index = torch.cat([edge_index.flip(0)], dim=1)
            edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

            node_features = sample[:, 1:3]  # z. B. Koordinaten

            z = F.relu(self.conv1(node_features, edge_index))
            #z = F.relu(self.conv2(z, edge_index))  # Optional: weitere Aktivierung

            logits = self.fc_out(z)               # [num_nodes, 2]
            probs = torch.sigmoid(logits)         # Binäre Wahrscheinlichkeiten für beide Labels
            
            outputs.append(probs)

        return torch.stack(outputs, dim=0)  # [B, num_nodes, 2]

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

def train_gcn_model(targets, inputs, model = None, num_epochs=30, batch_size=1, lr=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = TensorDataset(inputs, targets)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    if model == None:
        model = GCNEmbedding()
    model.to(device)

    # pos_weight berechnen für jede Klasse (Shape: [2])
    total_positives = targets.sum(dim=(0, 1))  # shape: [2]
    total_negatives = targets.shape[0] * targets.shape[1] - total_positives
    pos_weight = total_negatives / (total_positives + 1e-6)  # Numerische Stabilität
    pos_weight = torch.sqrt(total_negatives / (total_positives + 1e-8))
    pos_weight = pos_weight.clamp(min=1.0, max=100.0).to(device)

    print(f"pos_weight used for BCEWithLogitsLoss: {pos_weight}")

    loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        all_preds = []
        all_labels = []

        for xb, yb in dataloader:
            xb = xb.to(device)
            yb = yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)  # [B, N, 2]
            loss = loss_fn(logits, yb.float())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Zum Debuggen: Prediction & Label sammeln
            probs = torch.sigmoid(logits.detach())
            preds = (probs > 0.5).float()
            all_preds.append(preds.cpu())
            all_labels.append(yb.cpu())

        avg_loss = total_loss / len(dataloader)

        # Accuracy & co. berechnen
        all_preds = torch.cat(all_preds, dim=0).view(-1, 1)
        all_labels = torch.cat(all_labels, dim=0).view(-1, 1)
        correct = (all_preds == all_labels).float()
        acc = correct.mean().item()
        precision = (all_preds * all_labels).sum(dim=0) / (all_preds.sum(dim=0) + 1e-6)
        recall = (all_preds * all_labels).sum(dim=0) / (all_labels.sum(dim=0) + 1e-6)

        print(f"Epoch {epoch+1:02d}, Loss: {avg_loss:.4f}, Acc: {acc:.3f}, Prec: {precision}, Recall: {recall}")

    return model


model = train_gcn_model(theta_all, x_all_flat, model = None, num_epochs=60, batch_size=32)


pos_weight used for BCEWithLogitsLoss: tensor([14.0712])
Epoch 01, Loss: 0.9890, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 02, Loss: 0.9890, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 03, Loss: 0.9889, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 04, Loss: 0.9889, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 05, Loss: 0.9889, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 06, Loss: 0.9888, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 07, Loss: 0.9888, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 08, Loss: 0.9887, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 09, Loss: 0.9887, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 10, Loss: 0.9886, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 11, Loss: 0.9886, Acc: 0.005, Prec: tensor([0.0050]), Recall: tensor([1.0000])
Epoch 12

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# ===================================
# Hilfsfunktion: Bias-Initialisierung
# ===================================
def init_last_bias_to_prior(model, prior_pos):
    """
    Setzt den Bias der letzten Linearschicht auf log(p/(1-p)).
    prior_pos: Tensor [C], P(y=1) pro Klasse.
    """
    p = torch.clamp(prior_pos, 1e-6, 1 - 1e-6)
    bias = torch.log(p / (1 - p))
    last_linear = None
    for m in reversed(list(model.modules())):
        if isinstance(m, nn.Linear) and m.out_features == prior_pos.numel():
            last_linear = m
            break
    if last_linear is not None:
        with torch.no_grad():
            last_linear.bias.copy_(bias)
        print(f"Initialized last layer bias to {bias}")
    else:
        print("WARNING: Could not find last linear layer for bias init.")


# ===================================
# Training
# ===================================
def train_gcn_model(
    targets,
    inputs,
    model=None,
    num_epochs=30,
    batch_size=16,
    lr=1e-4,
    use_temperature=False,
    temperature=1.5,  # Skaliert Logits beim Eval
    min_threshold=0.05
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = TensorDataset(inputs, targets)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    if model is None:
        model = GCNEmbedding()
    model.to(device)

    # ==========================
    # Klassenprävalenz & pos_weight
    # ==========================
    with torch.no_grad():
        total = targets.shape[0] * targets.shape[1]
        total_positives = targets.sum(dim=(0, 1))  # [C]
        total_negatives = total - total_positives
        pos_weight = (total_negatives / (total_positives + 1e-6)).clamp(1.0, 20.0)
        pos_weight = pos_weight.to(device)
        prior_pos = total_positives / total
        print(f"pos_weight used for BCEWithLogitsLoss: {pos_weight}")

    # Bias-Init
    init_last_bias_to_prior(model, prior_pos.to(device))

    loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for xb, yb in dataloader:
            xb = xb.to(device)
            yb = yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)  # [B, N, C]
            loss = loss_fn(logits, yb.float())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)

        # ==========================
        # Evaluation (Train-Set)
        # ==========================
        model.eval()
        with torch.no_grad():
            xb = inputs.to(device)
            yb = targets.to(device)

            logits = model(xb)
            if use_temperature:
                logits = logits * temperature

            probs = torch.sigmoid(logits).cpu()
            labels = yb.cpu()

            # Metrics
            flat_probs = probs.view(-1, probs.shape[-1])
            flat_labels = labels.view(-1, labels.shape[-1])

            thresholds, precisions, recalls = [], [], []

            for k in range(flat_probs.shape[1]):
                values = flat_probs[:, k]
                ts = torch.quantile(values, torch.linspace(0, 1, 51))
                best_f1, best_t = 0.0, 0.5

                for t in ts:
                    t = max(min_threshold, float(t))
                    preds = (values > t).float()
                    tp = (preds * flat_labels[:, k]).sum()
                    fp = (preds * (1 - flat_labels[:, k])).sum()
                    fn = ((1 - preds) * flat_labels[:, k]).sum()
                    precision = tp / (tp + fp + 1e-8)
                    recall = tp / (tp + fn + 1e-8)
                    f1 = 2 * precision * recall / (precision + recall + 1e-8)
                    if f1 > best_f1:
                        best_f1, best_t = f1.item(), t

                thresholds.append(best_t)
                preds_best = (values > best_t).float()
                tp = (preds_best * flat_labels[:, k]).sum()
                fp = (preds_best * (1 - flat_labels[:, k])).sum()
                fn = ((1 - preds_best) * flat_labels[:, k]).sum()
                precisions.append((tp / (tp + fp + 1e-8)).item())
                recalls.append((tp / (tp + fn + 1e-8)).item())

            print(f"Epoch {epoch+1:02d}, Loss: {avg_loss:.4f}, "
                  f"Thresholds: {thresholds}, Prec: {precisions}, Rec: {recalls}")

    return model

model = train_gcn_model(theta_all, x_all_flat, model = None, num_epochs=60, batch_size=16)

pos_weight used for BCEWithLogitsLoss: tensor([20.])
Initialized last layer bias to tensor([-5.2883])
Epoch 01, Loss: 0.7617, Thresholds: [0.5], Prec: [0.005025125574320555], Rec: [1.0]
Epoch 02, Loss: 0.7617, Thresholds: [0.5], Prec: [0.005025125574320555], Rec: [1.0]
Epoch 03, Loss: 0.7617, Thresholds: [0.5], Prec: [0.005025125574320555], Rec: [1.0]
Epoch 04, Loss: 0.7617, Thresholds: [0.5], Prec: [0.005025125574320555], Rec: [1.0]
Epoch 05, Loss: 0.7617, Thresholds: [0.5], Prec: [0.005025125574320555], Rec: [1.0]
Epoch 06, Loss: 0.7617, Thresholds: [0.5], Prec: [0.005025125574320555], Rec: [1.0]
Epoch 07, Loss: 0.7617, Thresholds: [0.5002151727676392], Prec: [0.005050505045801401], Rec: [1.0]
Epoch 08, Loss: 0.7617, Thresholds: [0.5002182126045227], Prec: [0.005050505045801401], Rec: [1.0]
Epoch 09, Loss: 0.7617, Thresholds: [0.500221312046051], Prec: [0.005050505045801401], Rec: [1.0]
Epoch 10, Loss: 0.7617, Thresholds: [0.5002245306968689], Prec: [0.005050505045801401], Rec: [1.0]

In [9]:
print(model)

GCNEmbedding(
  (conv1): GCNConv(2, 32)
  (conv2): GCNConv(32, 32)
  (fc_out): Linear(in_features=32, out_features=1, bias=True)
)
